In [1]:
import numpy as np                                          # 导入numpy库并简写为np
import pandas as pd
import itertools
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import argparse
from tqdm import tqdm
from QAOA_CSP import CSP, _H,getBaseProb

def I(n): #返回可占用n个比特位的I矩阵
    return np.eye(2**n)
z = np.array([[1,0],
                [0,-1]],dtype=np.complex128)
def create_new_dict(nested_dict, func) -> dict:
    new_dict = {}
    for key, value in nested_dict.items():
        if isinstance(value, dict):
            new_dict[key] = create_new_dict(value, func)
        else:
            new_key, new_value = func(key, value)
            new_dict[new_key] = new_value
    
    return new_dict

# 将二层的key转化为我要的形式的字符串
def modify_key_value(key, value):
    operLst = ",".join(list(map(lambda x:str(eval(x[2]) - 1),key.split())))

    return operLst, value
def Qaoa3E(Problem:dict,HC,gammaLst:list,betaLst:list,p:int,s:np.ndarray) -> float:
    model = CSP(Problem,HC)
    model.updateQC(gammaLst,betaLst,p)
    return model.getExpctation(s)

# Problem转为经典表达式
def paraCSP(Problem,z1,z2,z3,z4):
    C = 0
    Problem_beta = Problem[1]
    Constant = Problem[2]
    factor = Problem[3]
    for _, Power in Problem_beta.items(): #每个次幂的
        for key, coef in Power.items(): #某个次幂中 每个项
            # 解析每个项的表达式，并乘value系数
            item = 1
            for oper in key.split(): 
                item  = item * eval("z{}".format(oper[2]))
            C  = C + item*coef 
    return factor*(C+Constant)
def GridSearch_CSP(Problem):
    sulotionsSet = itertools.product([-1,1],repeat=4)
    obj_maxLst = [0]
    para_max = []
    for z1,z2,z3,z4 in sulotionsSet:
        obj = paraCSP(Problem,z1,z2,z3,z4)
        if obj > obj_maxLst[0]:
            obj_maxLst = [obj]
            para_max = [(z1,z2,z3,z4)]
        elif obj == obj_maxLst[0]:
            obj_maxLst.append(obj)
            para_max.append((z1,z2,z3,z4))
        else:
            pass
    return obj_maxLst,para_max
# 结果评价指标 信息熵

def Entropy(probabilities:np.ndarray):
    # 计算数组中每个元素的概率
    # 计算信息熵
    entropy = -np.sum(probabilities * np.log(probabilities))
    # 归一化信息熵
    n_entropy = 1 - (entropy / np.log(probabilities.size))
    
    return n_entropy


In [2]:
# 串行版本 计算每个formula的情况
combinations = itertools.product([0, -1, 1], repeat=15)
CtrlSeries_Lst = [c for c in combinations if sum(1 for i in c[6:] if i != 0) <= 1][-10000:]

# 串行版本 计算每个formula的情况
df_CSP = pd.DataFrame(data = {"E":np.nan,
                              "solutions":np.nan,
                              "Entropy":np.nan,
                              "gamma_1":np.nan,
                              "gamma_2":np.nan,
                              "beta_1":np.nan,
                              "beta_2":np.nan,
                              "formula":CtrlSeries_Lst})
Constant = 0
factor = 1
N = 15 #采样数
p = 1 # p = 1
t = np.concatenate((np.array([1]) , np.zeros((15)))).reshape(-1,1) #创建初态
s = (_H() + _H() + _H() + _H()).dot(t).to_array()
gamma_1Lst = np.linspace(0,2*np.pi,N*2) #生成遍历参数列表
beta_1Lst = np.linspace(0,np.pi,N) 
#beta_2Lst = np.linspace(0,np.pi,N)
#gamma_2Lst = np.linspace(0,2*np.pi,N*2)
#*********** 并行版本 ******************
pbar = tqdm(total=df_CSP.shape[0],desc = "Process: ")
for idx,row in tqdm(df_CSP.iterrows()):
    #----------------- 初始化Problem
    CtrlSeries = row["formula"]
    Power_1 = {"Z_1":CtrlSeries[0], "Z_2":CtrlSeries[1], "Z_3":CtrlSeries[2],"Z_4":CtrlSeries[3]}
    Power_2 = {"Z_1 Z_2":CtrlSeries[4], "Z_3 Z_4":CtrlSeries[5],
            "Z_1 Z_3":CtrlSeries[6], "Z_1 Z_4":CtrlSeries[7], "Z_2 Z_3":CtrlSeries[8], "Z_2 Z_4":CtrlSeries[9]}
    Power_3 = {"Z_1 Z_2 Z_3":CtrlSeries[10], "Z_1 Z_2 Z_4":CtrlSeries[11], "Z_1 Z_3 Z_4": CtrlSeries[12] , "Z_2 Z_3 Z_4":CtrlSeries[13]}
    Power_4 = {"Z_1 Z_2 Z_3 Z_4":CtrlSeries[14]}
    Problem_beta = {"Power_1":Power_1,
            "Power_2":Power_2,
            "Power_3":Power_3,
            "Power_4":Power_4}
    Problem_alpha = create_new_dict(Problem_beta , modify_key_value)
    Problem = (Problem_alpha,Problem_beta, Constant, factor)
    #----------------- 获取CSP(Problem)
    model = CSP(Problem)
    HC = model.HC

    loopSeries = itertools.product(gamma_1Lst,beta_1Lst)

    E_lst = Parallel(n_jobs=4,verbose=0)(delayed(Qaoa3E)(Problem,HC,[gamma_1],[beta_1],p,s) 
                    for gamma_1,beta_1 in list(loopSeries))
    df = pd.DataFrame(list(itertools.product(gamma_1Lst,beta_1Lst)), columns=["gamma_1","beta_1",]) #得到参数的数据框
    df.insert(0,column="E",value = E_lst) #将E_lst插入数据框
    row = df.loc[df["E"].idxmax(),:] # E_max对应列
    model.updateQC([row["gamma_1"]],[row["beta_1"]],p)
    pureState = model.getState(s) # 返回该最优量子线路输出的纯态
    State = getBaseProb(pureState).real #得到概率向量
    idxMax = np.argmax(State) #binary变量为 二进制编码的解
    binary = bin(idxMax)[2:].zfill(4)
    
    #-------------保存信息熵和期望
    df_CSP.at[idx,"E"] = row["E"]
    df_CSP.at[idx,"solutions"] = binary
    df_CSP.at[idx,"Entropy"] = Entropy(State)
    df_CSP.at[idx,"gamma_1"] = row["gamma_1"]
    df_CSP.at[idx,"beta_1"] = row["beta_1"]
    pbar.update(1)
    break

0it [00:00, ?it/s]      | 1/10000 [00:00<2:18:17,  1.21it/s]


In [8]:
df_CSP["formula"][0]

(0, 1, -1, -1, -1, -1, 0, 0, -1, 0, 0, 0, 0, 0, 0)

In [3]:
Problem

({'Power_1': {'0': 0, '1': 1, '2': -1, '3': -1},
  'Power_2': {'0,1': -1, '2,3': -1, '0,2': 0, '0,3': 0, '1,2': -1, '1,3': 0},
  'Power_3': {'0,1,2': 0, '0,1,3': 0, '0,2,3': 0, '1,2,3': 0},
  'Power_4': {'0,1,2,3': 0}},
 {'Power_1': {'Z_1': 0, 'Z_2': 1, 'Z_3': -1, 'Z_4': -1},
  'Power_2': {'Z_1 Z_2': -1,
   'Z_3 Z_4': -1,
   'Z_1 Z_3': 0,
   'Z_1 Z_4': 0,
   'Z_2 Z_3': -1,
   'Z_2 Z_4': 0},
  'Power_3': {'Z_1 Z_2 Z_3': 0,
   'Z_1 Z_2 Z_4': 0,
   'Z_1 Z_3 Z_4': 0,
   'Z_2 Z_3 Z_4': 0},
  'Power_4': {'Z_1 Z_2 Z_3 Z_4': 0}},
 0,
 1)

In [15]:
def process_p1(row,p): #并行计算p=1时每种问题的情况
    CtrlSeries = row["formula"]
    Power_1 = {"Z_1":CtrlSeries[0], "Z_2":CtrlSeries[1], "Z_3":CtrlSeries[2],"Z_4":CtrlSeries[3]}
    Power_2 = {"Z_1 Z_2":CtrlSeries[4], "Z_3 Z_4":CtrlSeries[5],
            "Z_1 Z_3":CtrlSeries[6], "Z_1 Z_4":CtrlSeries[7], "Z_2 Z_3":CtrlSeries[8], "Z_2 Z_4":CtrlSeries[9]}
    Power_3 = {"Z_1 Z_2 Z_3":CtrlSeries[10], "Z_1 Z_2 Z_4":CtrlSeries[11], "Z_1 Z_3 Z_4": CtrlSeries[12] , "Z_2 Z_3 Z_4":CtrlSeries[13]}
    Power_4 = {"Z_1 Z_2 Z_3 Z_4":CtrlSeries[14]}
    Problem_beta = {"Power_1":Power_1,
            "Power_2":Power_2,
            "Power_3":Power_3,
            "Power_4":Power_4}
    Problem_alpha = create_new_dict(Problem_beta , modify_key_value)
    Problem = (Problem_alpha,Problem_beta, Constant, factor)
    #----------------- 获取CSP(Problem)
    model = CSP(Problem)
    HC = model.HC
    loopSeries = list(itertools.product(gamma_1Lst,beta_1Lst))
    E_lst = Parallel(n_jobs=1,verbose=0)(delayed(Qaoa3E)(Problem,HC,[gamma_1],[beta_1],p,s) 
                    for gamma_1,beta_1 in loopSeries)
    df = pd.DataFrame(loopSeries, columns=["gamma_1","beta_1",]) #得到参数的数据框
    df.insert(0,column="E",value = E_lst) #将E_lst插入数据框
    row = df.loc[df["E"].idxmax(),:] # E_max对应列
    model.updateQC([row["gamma_1"]],[row["beta_1"]],p)
    pureState = model.getState(s) # 返回该最优量子线路输出的纯态
    State = getBaseProb(pureState).real #得到概率向量
    idxMax = np.argmax(State) #binary变量为 二进制编码的解
    binary = bin(idxMax)[2:].zfill(4)
    #--------------- 上述为量子部分，下述为经典网格搜索部分
    ObjLst,Num_solution_Lst = GridSearch_CSP(Problem)
    New_Num_solution_Lst = []
    for lt in Num_solution_Lst:
        New_Num_solution_Lst.append(''.join(str(int((1-x)/2)) for x in lt))
    #------------输出期望，QAOA的最优解，概率向量的信息熵，gamma_1,beta_1, 经典最优目标值向量(可能由多解)，经典最优解向量，QAOA的解是否为最优解之一
    return row["E"],binary,Entropy(State),row["gamma_1"],row["beta_1"],ObjLst,New_Num_solution_Lst,binary in New_Num_solution_Lst

def process_p2(row,p): #并行计算p=2时每种问题的情况
    CtrlSeries = row["formula"]
    Power_1 = {"Z_1":CtrlSeries[0], "Z_2":CtrlSeries[1], "Z_3":CtrlSeries[2],"Z_4":CtrlSeries[3]}
    Power_2 = {"Z_1 Z_2":CtrlSeries[4], "Z_3 Z_4":CtrlSeries[5],
            "Z_1 Z_3":CtrlSeries[6], "Z_1 Z_4":CtrlSeries[7], "Z_2 Z_3":CtrlSeries[8], "Z_2 Z_4":CtrlSeries[9]}
    Power_3 = {"Z_1 Z_2 Z_3":CtrlSeries[10], "Z_1 Z_2 Z_4":CtrlSeries[11], "Z_1 Z_3 Z_4": CtrlSeries[12] , "Z_2 Z_3 Z_4":CtrlSeries[13]}
    Power_4 = {"Z_1 Z_2 Z_3 Z_4":CtrlSeries[14]}
    Problem_beta = {"Power_1":Power_1,
            "Power_2":Power_2,
            "Power_3":Power_3,
            "Power_4":Power_4}
    Problem_alpha = create_new_dict(Problem_beta , modify_key_value)
    Problem = (Problem_alpha,Problem_beta, Constant, factor)
    #----------------- 获取CSP(Problem)
    model = CSP(Problem)
    HC = model.HC
    loopSeries = list(itertools.product(gamma_1Lst,gamma_1Lst,beta_1Lst,beta_1Lst))
    E_lst = Parallel(n_jobs=1,verbose=0)(delayed(Qaoa3E)(Problem,HC,[gamma_1,gamma_2],[beta_1,beta_2],p,s) 
                    for gamma_1,gamma_2,beta_1,beta_2 in loopSeries)
    df = pd.DataFrame(loopSeries, columns=["gamma_1","gamma_2","beta_1","beta_2"]) #得到参数的数据框
    df.insert(0,column="E",value = E_lst) #将E_lst插入数据框
    row = df.loc[df["E"].idxmax(),:] # E_max对应列
    model.updateQC([row["gamma_1"],row["gamma_2"]],[row["beta_1"],row["beta_2"]],p)
    pureState = model.getState(s) # 返回该最优量子线路输出的纯态
    State = getBaseProb(pureState).real #得到概率向量
    idxMax = np.argmax(State) #binary变量为 二进制编码的解
    binary = bin(idxMax)[2:].zfill(4)
    #--------------- 上述为量子部分，下述为经典网格搜索部分
    ObjLst,Num_solution_Lst = GridSearch_CSP(Problem)
    New_Num_solution_Lst = []
    for lt in Num_solution_Lst:
        New_Num_solution_Lst.append(''.join(str(int((1-x)/2)) for x in lt))
    #------------输出期望，QAOA的最优解，概率向量的信息熵，gamma_1,gamma_2, beta_1,beta_2, 经典最优目标值向量(可能由多解)，经典最优解向量，QAOA的解是否为最优解之一
    return row["E"],binary,Entropy(State),row["gamma_1"],row["gamma_2"],row["beta_1"],row["beta_2"],ObjLst,New_Num_solution_Lst,binary in New_Num_solution_Lst

def process_realSulotion(idx,row,df_CSP): #暴力搜索每种问题，没什么卵用，先留着
    CtrlSeries = row["formula"]
    Power_1 = {"Z_1":CtrlSeries[0], "Z_2":CtrlSeries[1], "Z_3":CtrlSeries[2],"Z_4":CtrlSeries[3]}
    Power_2 = {"Z_1 Z_2":CtrlSeries[4], "Z_3 Z_4":CtrlSeries[5],
            "Z_1 Z_3":CtrlSeries[6], "Z_1 Z_4":CtrlSeries[7], "Z_2 Z_3":CtrlSeries[8], "Z_2 Z_4":CtrlSeries[9]}
    Power_3 = {"Z_1 Z_2 Z_3":CtrlSeries[10], "Z_1 Z_2 Z_4":CtrlSeries[11], "Z_1 Z_3 Z_4": CtrlSeries[12] , "Z_2 Z_3 Z_4":CtrlSeries[13]}
    Power_4 = {"Z_1 Z_2 Z_3 Z_4":CtrlSeries[14]}
    Problem_beta = {"Power_1":Power_1,
            "Power_2":Power_2,
            "Power_3":Power_3,
            "Power_4":Power_4}
    Problem_alpha = create_new_dict(Problem_beta , modify_key_value)
    Problem = (Problem_alpha,Problem_beta, Constant, factor)
    ObjLst,Num_solution_Lst = GridSearch_CSP(Problem)
    New_Num_solution_Lst = []
    for lt in Num_solution_Lst:
        New_Num_solution_Lst.append(''.join(str(int((1-x)/2)) for x in lt))
    return ObjLst,New_Num_solution_Lst
    #return ''.join(str(int(1-x)/2) for x in Num_solution)

In [27]:
combinations = itertools.product([0, -1, 1], repeat=15)
CtrlSeries_Lst = [c for c in combinations if sum(1 for i in c[6:] if i != 0) == 1]


In [ ]:
# p=1的遍历
df_CSP = pd.DataFrame(data = {"E":np.nan,
                              "solutions":np.nan,
                              "real_solutions":np.nan,
                              "isOpt":np.nan,
                              "Entropy":np.nan,
                              "gamma_1":np.nan,
                              "gamma_2":np.nan,
                              "beta_1":np.nan,
                              "beta_2":np.nan,
                              "formula":CtrlSeries_Lst})
Constant = 0
factor = 1
N = 15 #采样数
p = 1 # p = 1
jobs = 12
t = np.concatenate((np.array([1]) , np.zeros((15)))).reshape(-1,1) #创建初态
s = (_H() + _H() + _H() + _H()).dot(t).to_array()
gamma_1Lst = np.linspace(0,2*np.pi,N*2) #生成遍历参数列表
beta_1Lst = np.linspace(0,np.pi,N) 
fill_data = Parallel(n_jobs=jobs,verbose=0)(delayed(process_p1)(row,p) 
                    for _,row in tqdm(list(df_CSP.iterrows())))
fill_df = pd.DataFrame(data = fill_data,columns = ["E", "solutions", "Entropy","gamma_1", "beta_1","MaxObj","real_solutions","isOpt"])
for col in fill_df: #将输出填入df_CSP数据框中
    df_CSP[col] = fill_df[col]
tmpLst = df_CSP.apply(lambda row: row["E"]/ row["MaxObj"][0],axis = 1)
df_CSP.insert(1,"ratio",value=tmpLst)

In [ ]:
# p=2的遍历

df_CSP = pd.DataFrame(data = {"E":np.nan,
                              "solutions":np.nan,
                              "real_solutions":np.nan,
                              "isOpt":np.nan,
                              "Entropy":np.nan,
                              "gamma_1":np.nan,
                              "gamma_2":np.nan,
                              "beta_1":np.nan,
                              "beta_2":np.nan,
                              "formula":CtrlSeries_Lst})
Constant = 0
factor = 1
N = 5 #采样数
p = 2 # p = 2
jobs = 12
t = np.concatenate((np.array([1]) , np.zeros((15)))).reshape(-1,1) #创建初态
s = (_H() + _H() + _H() + _H()).dot(t).to_array()
gamma_1Lst = np.linspace(0,2*np.pi,N*2) #生成遍历参数列表
beta_1Lst = np.linspace(0,np.pi,N) 
fill_data = Parallel(n_jobs=jobs,verbose=0)(delayed(process_p2)(row,p) 
                    for _,row in tqdm(list(df_CSP.iterrows())))
fill_df = pd.DataFrame(data = fill_data,columns = ["E", "solutions", "Entropy","gamma_1","gamma_2", "beta_1", "beta_2", "MaxObj","real_solutions","isOpt"])
for col in fill_df: #将输出填入df_CSP数据框中
    df_CSP[col] = fill_df[col]
tmpLst = df_CSP.apply(lambda row: row["E"]/ row["MaxObj"][0],axis = 1)
df_CSP.insert(1,"ratio",value=tmpLst)

In [10]:
df_CSP_1 = pd.read_csv("df_CSP_p1_External=1.csv",index_col=0, dtype={'solutions':str})

In [11]:
df_CSP_2 = pd.read_csv("df_CSP_p1_External=2.csv",index_col=0,dtype={'solutions':str})

In [11]:
for idx,row in df_CSP.iterrows(): # 串行测试
    process_p2(row)
    break

In [19]:
'''    p = 2的模板
model = CSP(Problem)
    HC = model.HC
    N = 20 #采样数
    t = np.concatenate((np.array([1]) , np.zeros((15)))).reshape(-1,1) #创建初态
    s = (_H() + _H() + _H() + _H()).dot(t).to_array()
    beta_1Lst = np.linspace(0,np.pi,N) #生成遍历参数列表
    gamma_1Lst = np.linspace(0,2*np.pi,N*2) 
    beta_2Lst = np.linspace(0,np.pi,N)
    gamma_2Lst = np.linspace(0,2*np.pi,N*2)
    #*********** 并行版本 ******************
    loopSeries = itertools.product(gamma_1Lst,gamma_2Lst,beta_1Lst,beta_2Lst)
    E_lst = Parallel(n_jobs=8,verbose=0)(delayed(Qaoa3E)(Problem,HC,[gamma_1,gamma_2],[beta_1,beta_2],s) 
                    for gamma_1,gamma_2,beta_1,beta_2 in tqdm(list(loopSeries)))
    df = pd.DataFrame(list(itertools.product(gamma_1Lst,gamma_2Lst,beta_1Lst,beta_2Lst)), columns=["gamma_1","gamma_2","beta_1","beta_2"]) #得到参数的数据框
    df.insert(0,column="E",value = E_lst) #将E_lst插入数据框
    row = df.loc[df["E"].idxmax(),:] # E_max对应列
    model.updateQC((row["gamma_1"],row["gamma_2"]),(row["beta_1"],row["beta_2"]),2)
    pureState = model.getState(s) # 返回该最优量子线路输出的纯态
    State = getBaseProb(pureState)
    df_CSP.at[idx,"Entropy"] = Entropy(State)
    df_CSP.at[idx,"E"] = row["E"]
'''

'    p = 2的模板\nmodel = CSP(Problem)\n    HC = model.HC\n    N = 20 #采样数\n    t = np.concatenate((np.array([1]) , np.zeros((15)))).reshape(-1,1) #创建初态\n    s = (_H() + _H() + _H() + _H()).dot(t).to_array()\n    beta_1Lst = np.linspace(0,np.pi,N) #生成遍历参数列表\n    gamma_1Lst = np.linspace(0,2*np.pi,N*2) \n    beta_2Lst = np.linspace(0,np.pi,N)\n    gamma_2Lst = np.linspace(0,2*np.pi,N*2)\n    #*********** 并行版本 ******************\n    loopSeries = itertools.product(gamma_1Lst,gamma_2Lst,beta_1Lst,beta_2Lst)\n    E_lst = Parallel(n_jobs=8,verbose=0)(delayed(Qaoa3E)(Problem,HC,[gamma_1,gamma_2],[beta_1,beta_2],s) \n                    for gamma_1,gamma_2,beta_1,beta_2 in tqdm(list(loopSeries)))\n    df = pd.DataFrame(list(itertools.product(gamma_1Lst,gamma_2Lst,beta_1Lst,beta_2Lst)), columns=["gamma_1","gamma_2","beta_1","beta_2"]) #得到参数的数据框\n    df.insert(0,column="E",value = E_lst) #将E_lst插入数据框\n    row = df.loc[df["E"].idxmax(),:] # E_max对应列\n    model.updateQC((row["gamma_1"],row["gamma_2"